In [ ]:
import pyAgrum as gum
import pyAgrum.lib.notebook as gnb

In [ ]:
dn = gum.InfluenceDiagram()

In [ ]:
# Decision Nodes
call_check_in=dn.addDecisionNode(gum.LabelizedVariable('CallCheckIn', 'call_check_in', ['no', 'yes']))

# Chance Nodes
# government_support=dn.addChanceNode(gum.LabelizedVariable('GovernmentSupport', 'government_support', ['no', 'yes']))
# school_motivation=dn.addChanceNode(gum.LabelizedVariable('SchoolMotivation','schoolmotivation', ['low', 'medium', 'high']))
# academic_performance=dn.addChanceNode(gum.LabelizedVariable('AcademicPerformance','academic_performance', ['0-50', '50-60', '70-100']))
# attendance=dn.addChanceNode(gum.LabelizedVariable('Attendance','attendance', ['low', 'high']))
# family_support=dn.addChanceNode(gum.LabelizedVariable('FamilySupport', 'family_support', ['no', 'yes']))
# internet=dn.addChanceNode(gum.LabelizedVariable('Internet', 'internet',['no','yes']))
# family_income=dn.addChanceNode(gum.LabelizedVariable('FamilyIncome', 'family_income',['0-350000','350001-700000','700000+']))
# financial_support=dn.addChanceNode(gum.LabelizedVariable('FinancialSupport', 'financial_support', ['no', 'yes']))
# likelihood_of_dropout=dn.addChanceNode(gum.LabelizedVariable('LikelihoodOfDropout', 'likelihood_of_dropout', ['low', 'medium', 'high']))

# Chance Nodes
user_behaviour=dn.addChanceNode(gum.LabelizedVariable('UserBehavior','user_behaviour', ['low', 'medium', 'high']))
account_age=dn.addChanceNode(gum.LabelizedVariable('AccountAge','account_age', ['0-50', '50-60', '70-100']))
georgraphic_location=dn.addChanceNode(gum.LabelizedVariable('GeographicLocation', 'georgraphic_location', ['no', 'yes']))
transaction_amount=dn.addChanceNode(gum.LabelizedVariable('TransactionAmount', 'transaction_amount',['0-350000','350001-700000','700000+']))
transaction_info=dn.addChanceNode(gum.LabelizedVariable('TransactionInfo', 'transaction_info', ['no', 'yes']))
transaction_frequency=dn.addChanceNode(gum.LabelizedVariable('TransactionFrequency', 'transaction_frequency', ['no', 'yes']))
likelihood_of_fraud=dn.addChanceNode(gum.LabelizedVariable('LikelihoodOfFraud', 'likelihood_of_fraud', ['low', 'medium', 'high']))
print ("BN With vars : ",dn)


# Utility Nodes
check_in_needed = dn.addUtilityNode(gum.LabelizedVariable('CallCheckInNeeded', 'check_in_needed', 1))

In [ ]:
# Addding arcs between the nodes
# arcs = [(school_motivation, likelihood_of_dropout),(academic_performance, school_motivation), (attendance, school_motivation), (family_support, likelihood_of_dropout), (financial_support, likelihood_of_dropout), (government_support, financial_support), (family_income, financial_support),(internet, academic_performance),(school_work_shop, academic_performance),(school_work_shop, school_workshop_needed),(likelihood_of_dropout, school_workshop_needed),(family_income,school_workshop_needed)]#(government_support, grant_applicability) (likelihood_of_dropout, grant_applicability)
# for arc in arcs:
#     dn.addArc(*arc)
arcs = [(user_behaviour, likelihood_of_fraud),(account_age, user_behaviour), (transaction_info, likelihood_of_fraud), (transaction_frequency, transaction_info),(transaction_amount, transaction_info),(georgraphic_location, likelihood_of_fraud)]
for arc in arcs:
    dn.addArc(*arc)
dn

In [ ]:
# #setting probabilities
# dn.cpt('Attendance').fillWith([0.1,0.9])
# dn.cpt('Internet').fillWith([0.9,0.1])

# dn.cpt('GovernmentSupport').fillWith([0.92,0.08])
# dn.cpt('FamilyIncome').fillWith([0.3,0.4,0.2])
# dn.cpt('FamilySupport').fillWith([0.33,0.67])

# dn.cpt('AcademicPerformance')[{'SchoolWorkShop': 0, 'Internet': 0}] = [0.85, 0.1, 0.05]
# dn.cpt('AcademicPerformance')[{'SchoolWorkShop': 0, 'Internet': 1}] = [0.5, 0.25, 0.1]
# dn.cpt('AcademicPerformance')[{'SchoolWorkShop': 1, 'Internet': 0}] = [0.1, 0.4, 0.5]
# dn.cpt('AcademicPerformance')[{'SchoolWorkShop': 1, 'Internet': 1}] = [0.05, 0.55, 0.4]

# dn.cpt('SchoolMotivation')[{'AcademicPerformance': 0, 'Attendance': 0}] = [1, 0, 0]
# dn.cpt('SchoolMotivation')[{'AcademicPerformance': 0, 'Attendance': 1}] = [0.6, 0.15, 0.25]
# dn.cpt('SchoolMotivation')[{'AcademicPerformance': 1, 'Attendance': 0}] = [0.4, 0.5, 0.1]
# dn.cpt('SchoolMotivation')[{'AcademicPerformance': 1, 'Attendance': 1}] = [0.2, 0.525, 0.275]
# dn.cpt('SchoolMotivation')[{'AcademicPerformance': 2, 'Attendance': 0}] = [0.1, 0.675, 0.225]
# dn.cpt('SchoolMotivation')[{'AcademicPerformance': 2, 'Attendance': 1}] = [0.05, 0.8, 0.15]

# dn.cpt('FinancialSupport')[{'FamilyIncome': 0, 'GovernmentSupport': 0}] = [1, 0]
# dn.cpt('FinancialSupport')[{'FamilyIncome': 0, 'GovernmentSupport': 1}] = [0.35, 0.65]
# dn.cpt('FinancialSupport')[{'FamilyIncome': 1, 'GovernmentSupport': 0}] = [0.2, 0.8]
# dn.cpt('FinancialSupport')[{'FamilyIncome': 1, 'GovernmentSupport': 1}] = [0.1, 0.9]
# dn.cpt('FinancialSupport')[{'FamilyIncome': 2, 'GovernmentSupport': 0}] = [0.05, 0.95]
# dn.cpt('FinancialSupport')[{'FamilyIncome': 2, 'GovernmentSupport': 1}] = [0, 1]

# dn.cpt('LikelihoodOfDropout')[{'FamilySupport': 0, 'FinancialSupport': 0, 'SchoolMotivation': 0}] = [0.0, 0.45, 0.55]
# dn.cpt('LikelihoodOfDropout')[{'FamilySupport': 0, 'FinancialSupport': 0, 'SchoolMotivation': 1}] = [0.1, 0.5, 0.4]
# dn.cpt('LikelihoodOfDropout')[{'FamilySupport': 0, 'FinancialSupport': 0, 'SchoolMotivation': 2}] = [0.425, 0.325, 0.25]
# dn.cpt('LikelihoodOfDropout')[{'FamilySupport': 0, 'FinancialSupport': 1, 'SchoolMotivation': 0}] = [0.2, 0.5, 0.3]
# dn.cpt('LikelihoodOfDropout')[{'FamilySupport': 0, 'FinancialSupport': 1, 'SchoolMotivation': 1}] = [0.35, 0.5, 0.15]
# dn.cpt('LikelihoodOfDropout')[{'FamilySupport': 0, 'FinancialSupport': 1, 'SchoolMotivation': 2}] = [0.575, 0.325, 0.1] 
# dn.cpt('LikelihoodOfDropout')[{'FamilySupport': 1, 'FinancialSupport': 0, 'SchoolMotivation': 0}] = [0.4, 0.475, 0.125]
# dn.cpt('LikelihoodOfDropout')[{'FamilySupport': 1, 'FinancialSupport': 0, 'SchoolMotivation': 1}] = [0.6, 0.3, 0.1]
# dn.cpt('LikelihoodOfDropout')[{'FamilySupport': 1, 'FinancialSupport': 0, 'SchoolMotivation': 2}] = [.75, 0.155, 0.095]
# dn.cpt('LikelihoodOfDropout')[{'FamilySupport': 1, 'FinancialSupport': 1, 'SchoolMotivation': 0}] = [0.725, 0.225, 0.05]
# dn.cpt('LikelihoodOfDropout')[{'FamilySupport': 1, 'FinancialSupport': 1, 'SchoolMotivation': 1}] = [0.8, 0.15, 0.05]
# dn.cpt('LikelihoodOfDropout')[{'FamilySupport': 1, 'FinancialSupport': 1, 'SchoolMotivation': 2}] = [0.9, 0.08, 0.02]

# dn.utility('SchoolWorkshopNeeded')[{'FamilyIncome': 0,'LikelihoodOfDropout': 0, 'SchoolWorkShop': 0}] = 20
# dn.utility('SchoolWorkshopNeeded')[{'FamilyIncome': 0,'LikelihoodOfDropout': 0, 'SchoolWorkShop': 1}] = 18
# dn.utility('SchoolWorkshopNeeded')[{'FamilyIncome': 0,'LikelihoodOfDropout': 1, 'SchoolWorkShop': 0}] = 10
# dn.utility('SchoolWorkshopNeeded')[{'FamilyIncome': 0,'LikelihoodOfDropout': 1, 'SchoolWorkShop': 1}] = 8
# dn.utility('SchoolWorkshopNeeded')[{'FamilyIncome': 0,'LikelihoodOfDropout': 2, 'SchoolWorkShop': 0}] = 0
# dn.utility('SchoolWorkshopNeeded')[{'FamilyIncome': 0,'LikelihoodOfDropout': 2, 'SchoolWorkShop': 1}] = -2

# dn.utility('SchoolWorkshopNeeded')[{'FamilyIncome': 1,'LikelihoodOfDropout': 0, 'SchoolWorkShop': 0}] = 10
# dn.utility('SchoolWorkshopNeeded')[{'FamilyIncome': 1,'LikelihoodOfDropout': 0, 'SchoolWorkShop': 1}] = 8
# dn.utility('SchoolWorkshopNeeded')[{'FamilyIncome': 1,'LikelihoodOfDropout': 1, 'SchoolWorkShop': 0}] = 0
# dn.utility('SchoolWorkshopNeeded')[{'FamilyIncome': 1,'LikelihoodOfDropout': 1, 'SchoolWorkShop': 1}] = -2
# dn.utility('SchoolWorkshopNeeded')[{'FamilyIncome': 1,'LikelihoodOfDropout': 2, 'SchoolWorkShop': 0}] = -10
# dn.utility('SchoolWorkshopNeeded')[{'FamilyIncome': 1,'LikelihoodOfDropout': 2, 'SchoolWorkShop': 1}] = -12

# dn.utility('SchoolWorkshopNeeded')[{'FamilyIncome': 2,'LikelihoodOfDropout': 0, 'SchoolWorkShop': 0}] = 0
# dn.utility('SchoolWorkshopNeeded')[{'FamilyIncome': 2,'LikelihoodOfDropout': 0, 'SchoolWorkShop': 1}] = -2
# dn.utility('SchoolWorkshopNeeded')[{'FamilyIncome': 2,'LikelihoodOfDropout': 1, 'SchoolWorkShop': 0}] = -10
# dn.utility('SchoolWorkshopNeeded')[{'FamilyIncome': 2,'LikelihoodOfDropout': 1, 'SchoolWorkShop': 1}] = -12
# dn.utility('SchoolWorkshopNeeded')[{'FamilyIncome': 2,'LikelihoodOfDropout': 2, 'SchoolWorkShop': 0}] = -20
# dn.utility('SchoolWorkshopNeeded')[{'FamilyIncome': 2,'LikelihoodOfDropout': 2, 'SchoolWorkShop': 1}] = -22

# Define the conditional probability tables 

dn.cpt('AccountAge').fillWith([0.2,0.65,0.15])
dn.cpt('TransactionFrequency').fillWith([0.92,0.08])
dn.cpt('TransactionAmount').fillWith([0.3,0.4,0.2])

dn.cpt('TransactionInfo')[{'TransactionAmount': 0, 'TransactionFrequency': 0}] = [1, 0]
dn.cpt('TransactionInfo')[{'TransactionAmount': 0, 'TransactionFrequency': 1}] = [0.75, 0.25]
dn.cpt('TransactionInfo')[{'TransactionAmount': 1, 'TransactionFrequency': 0}] = [0.35, 0.65]
dn.cpt('TransactionInfo')[{'TransactionAmount': 1, 'TransactionFrequency': 1}] = [0.175, 0.825]
dn.cpt('TransactionInfo')[{'TransactionAmount': 2, 'TransactionFrequency': 0}] = [0.05, 0.95]
dn.cpt('TransactionInfo')[{'TransactionAmount': 2, 'TransactionFrequency': 1}] = [0, 1]

dn.cpt('GeographicLocation').fillWith([0.33,0.67])

dn.cpt('UserBehavior')[{'AccountAge': 0}] = [0.6, 0.15, 0.25]
dn.cpt('UserBehavior')[{'AccountAge': 1}] = [0.4, 0.5, 0.1]
dn.cpt('UserBehavior')[{'AccountAge': 2}] = [0.05, 0.8, 0.15]

dn.cpt('LikelihoodOfFraud')[{'GeographicLocation': 0, 'TransactionInfo': 0, 'UserBehavior': 0}] = [0.0, 0.45, 0.55]
dn.cpt('LikelihoodOfFraud')[{'GeographicLocation': 0, 'TransactionInfo': 0, 'UserBehavior': 1}] = [0.1, 0.5, 0.4]
dn.cpt('LikelihoodOfFraud')[{'GeographicLocation': 0, 'TransactionInfo': 0, 'UserBehavior': 2}] = [0.425, 0.325, 0.25]
dn.cpt('LikelihoodOfFraud')[{'GeographicLocation': 0, 'TransactionInfo': 1, 'UserBehavior': 0}] = [0.2, 0.5, 0.3]
dn.cpt('LikelihoodOfFraud')[{'GeographicLocation': 0, 'TransactionInfo': 1, 'UserBehavior': 1}] = [0.35, 0.5, 0.15]
dn.cpt('LikelihoodOfFraud')[{'GeographicLocation': 0, 'TransactionInfo': 1, 'UserBehavior': 2}] = [0.575, 0.325, 0.1]
dn.cpt('LikelihoodOfFraud')[{'GeographicLocation': 1, 'TransactionInfo': 0, 'UserBehavior': 0}] = [0.4, 0.475, 0.125]
dn.cpt('LikelihoodOfFraud')[{'GeographicLocation': 1, 'TransactionInfo': 0, 'UserBehavior': 1}] = [0.6, 0.3, 0.1]
dn.cpt('LikelihoodOfFraud')[{'GeographicLocation': 1, 'TransactionInfo': 0, 'UserBehavior': 2}] = [.75, 0.155, 0.095]
dn.cpt('LikelihoodOfFraud')[{'GeographicLocation': 1, 'TransactionInfo': 1, 'UserBehavior': 0}] = [0.825, 0.125, 0.05]
dn.cpt('LikelihoodOfFraud')[{'GeographicLocation': 1, 'TransactionInfo': 1, 'UserBehavior': 1}] = [0.915, 0.075, 0.01]
dn.cpt('LikelihoodOfFraud')[{'GeographicLocation': 1, 'TransactionInfo': 1, 'UserBehavior': 2}] = [0.955, 0.05, 0.005]

dn.utility('CallCheckInNeeded')[{'LikelihoodOfFraud': 0,'CallCheckIn': 0}] = 20
dn.utility('CallCheckInNeeded')[{'LikelihoodOfFraud': 0, 'CallCheckIn': 1}] = 12
dn.utility('CallCheckInNeeded')[{'LikelihoodOfFraud': 1, 'CallCheckIn': 0}] = 0
dn.utility('CallCheckInNeeded')[{'LikelihoodOfFraud': 1, 'CallCheckIn': 1}] = -2


In [ ]:
dn

In [ ]:
# With no evidence
gnb.showInference(dn, evs={})
dn.saveBIFXML("DN_scenario0.bif")

In [ ]:
#setting evidence
#scenario 1 (conditions applicable for workshop)
gnb.showInference(dn, evs={'FamilySupport':'no', 'Attendance':'low', 'Internet':'no','FamilyIncome':'0-350000'})
dn.saveBIFXML("DN_scenario1.bif")

In [ ]:
#scenario2 (evidence not applicable for workshop)
gnb.showInference(dn, evs={'FamilySupport':'yes', 'Attendance':'high','Internet': 'yes','FamilyIncome':'700000+'})
dn.saveBIFXML("DN_scenario2.bif")

In [ ]:
#scenario3 (evidence for an ambiguous decision)
gnb.showInference(dn, evs={'FamilySupport':'no', 'Attendance':'high','Internet': 'yes','FamilyIncome':'350001-700000'})
dn.saveBIFXML("DN_scenario3.bif")